In [26]:
import torch
from torch import nn

In [27]:
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] -h+1), (X.shape[1] -w +1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            mul = (X[i:i+h, j:j+w] * K)
#             print(mul)
            Y[i,j] = mul.sum()
    
    return Y

In [28]:
X = torch.tensor([[1.0,2.0,3.0], [2.0,3.0,4.0],[3.0,4.0,5.0]])
K = torch.tensor([[1.0,2.0], [2.0,3.0]])

corr2d(X,K)

tensor([[18., 26.],
        [26., 34.]])

In [29]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [30]:
net = Conv2d(K.shape)

In [31]:
net(X)

tensor([[-0.5964, -1.3331],
        [-1.3331, -2.0698]], grad_fn=<AddBackward0>)

### Edge detection

In [32]:
X = torch.ones((8,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [33]:
K = torch.tensor([[1.0,-1.0]])

In [34]:
corr2d(X,K)
# detects horizontal edges

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [35]:
corr2d(X.t(),K)
# but not vertical edges

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [36]:
X = torch.ones((8,8))

X[:,2:3] = 0
X[:,4:6] = 0

print(X)
Y = corr2d(X,K)
Y, Y.shape
# interesting

tensor([[1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.]])


(tensor([[ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.]]),
 torch.Size([8, 7]))

In [37]:
K.shape

torch.Size([1, 2])

### Learning a convolution

In [38]:
X = torch.ones((6,8))
X[:,2:6] = 0

Y =  corr2d(X,K)

In [39]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2), bias=False)

X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 3e-2

print(X.shape)
print(Y.shape)

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    
    if (i+1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum()}')


torch.Size([1, 1, 6, 8])
torch.Size([1, 1, 6, 7])
batch 2, loss 12.246731758117676
batch 4, loss 4.0018134117126465
batch 6, loss 1.468946099281311
batch 8, loss 0.5731261372566223
batch 10, loss 0.2299618273973465


In [40]:
conv2d.weight.data

tensor([[[[ 1.0385, -0.9402]]]])

### Exercises

1. Construct an image X with diagonal edges.
    1. What happens if you apply the kernel K in this section to it?
        * zero matrix.
    2. What happens if you transpose X?
        * No change
    3. What happens if you transpose K?
        * zero matrix.
        
2. When you try to automatically find the gradient for the Conv2D class we created, what kind
of error message do you see?
    * I am able to do `net.weights.grad`, when I try `net.grad` I get the error `'Conv2d' object has no attribute 'grad'`

3. How do you represent a cross-correlation operation as a matrix multiplication by changing
the input and kernel tensors?
    * cross correlation is basically matrix multiplication between slices of tensorfrom X of the shape of kernel and summing.
    * It can be done by padding Kand X based on what is needed to multiply

4. Design some kernels manually.
    1. What is the form of a kernel for the second derivative?
        * okay in order to compute one way would be to manually compute the second derivative and then let see a kernel be made using backpropogation
    2. What is the kernel for an integral?
        * how do you actually make it manually
3. What is the minimum size of a kernel to obtain a derivative of degree d
        * dont know.

In [41]:
#1

X =  torch.zeros((8,8))
for i in range(X.shape[0]):
    X[i][i] = 1
X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [42]:
K = torch.tensor([[1.0,-1.0]])
K.shape

torch.Size([1, 2])

In [43]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (Y[i:i+h, j : j +w] * K).sum()
    return Y

In [44]:
corr2d(X, K)

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [45]:
X.t(), X

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]))

In [46]:
corr2d(X, K.t())

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [47]:
Y = corr2d(X, K)
Y.shape

torch.Size([8, 7])

In [48]:
net = nn.Conv2d(1,1,K.shape, bias=False)

X = X.reshape(1,1,8,8)
Y = Y.reshape(1,1,8,7)

print(Y, X)
lr = 3e-2

for i in range(10):
    y_hat = net(X)
#     print(y_hat.shape)
    l = (y_hat - Y)**2
    
    net.zero_grad()
    
    l.sum().backward()
    
    net.weight.data -= lr * net.weight.grad
    
    print(f"for epoch {i} loss: {l.sum()}")


tensor([[[[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.]]]]) tensor([[[[1., 0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1.]]]])
for epoch 0 loss: 1.5457851886749268
for epoch 1 loss: 0.5200021266937256
for epoch 2 loss: 0.17492875456809998
for epoch 3 loss: 0.05884603410959244
for epoch 4 loss: 0.01979580894112587
for epoch 5 loss: 0.006659309379756451
for epoch 6 loss: 0.0022401916794478893
for epoch 7 loss: 0.0007536003831773996
for epoch 8 los

In [49]:
print(net.weight.data[0][0])
corr2d(X, net.weight.data[0][0])

tensor([[ 0.0008, -0.0018]])


tensor([], size=(1, 0))

In [50]:
try:
    net.grad
except Exception as e:
    print(e)

'Conv2d' object has no attribute 'grad'


### Strides and padding

In [51]:
import torch
from torch import nn

In [52]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [55]:
(1,1)+ X.shape

(1, 1, 6, 8)

In [61]:
Z = X.reshape((1,1) +X.shape)
Z
# 1 ,1 we add for batch and channel

tensor([[[[0.9080, 0.2015, 0.8998, 0.4439, 0.7717, 0.4837, 0.0411, 0.9311],
          [0.5910, 0.5630, 0.2239, 0.5713, 0.5696, 0.6923, 0.0312, 0.3428],
          [0.8309, 0.5663, 0.7552, 0.7841, 0.0527, 0.8695, 0.9922, 0.9605],
          [0.5985, 0.4539, 0.1946, 0.7523, 0.1578, 0.0621, 0.5854, 0.0146],
          [0.0588, 0.5586, 0.4790, 0.0938, 0.8431, 0.8413, 0.4735, 0.0134],
          [0.1204, 0.5902, 0.2952, 0.9974, 0.6445, 0.9038, 0.0709, 0.2500],
          [0.2286, 0.5401, 0.0302, 0.5498, 0.4925, 0.8790, 0.6081, 0.7232],
          [0.5011, 0.5229, 0.2495, 0.5114, 0.8894, 0.3726, 0.3668, 0.9662]]]])

In [57]:
conv2d(Z)

tensor([[[[ 0.0983,  1.0385,  0.0000,  0.0000,  0.0000, -0.9402,  0.0983],
          [ 0.0983,  1.0385,  0.0000,  0.0000,  0.0000, -0.9402,  0.0983],
          [ 0.0983,  1.0385,  0.0000,  0.0000,  0.0000, -0.9402,  0.0983],
          [ 0.0983,  1.0385,  0.0000,  0.0000,  0.0000, -0.9402,  0.0983],
          [ 0.0983,  1.0385,  0.0000,  0.0000,  0.0000, -0.9402,  0.0983],
          [ 0.0983,  1.0385,  0.0000,  0.0000,  0.0000, -0.9402,  0.0983]]]],
       grad_fn=<ThnnConv2DBackward>)

In [62]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1,1)+ X.shape)
    Y = conv2d(X)
    # we dont need batch and channels
    return Y.reshape(Y.shape[2:])

In [63]:
conv2d= nn.Conv2d(1,1,kernel_size=3, padding=1)
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape
# note that here since padding 1 means to either side 

torch.Size([8, 8])

In [64]:
conv2d= nn.Conv2d(1,1,kernel_size=(5,3), padding=(2,1))


comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [65]:
conv2d = nn.Conv2d(1,1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [71]:
conv2d = nn.Conv2d(1,1, kernel_size=(5,3), padding=(2,1), stride=2)
print(X.shape)
comp_conv2d(conv2d, X).shape

# it halves the input

torch.Size([8, 8])


torch.Size([4, 4])

In [77]:
conv2d = nn.Conv2d(1,1, kernel_size=(3,5), padding=(0,1), stride=(3,4))
conv2d(X.reshape((1,1)+ X.shape)).shape

torch.Size([1, 1, 2, 2])

### Exercises
1. For the last example in this section, use mathematics to calculate the output shape to see if
it is consistent with the experimental result.

* it is consistent, |(8 -3 + 0 + 3)/3| , |(8-5+1+4)/4|

2. Try other padding and stride combinations on the experiments in this section.

*  hmm tried

3. For audio signals, what does a stride of 2 correspond to?

* it might be two time peridod long

4. What are the computational benefits of a stride larger than 1

* efficiency in calculation,downsampling

##Multiple input and output channels